In [1]:
# Change directory to the root so that relative path loads work correctly
import os
try:
	os.chdir(os.path.join(os.getcwd(), '..'))
	print(os.getcwd())
except:
	pass

/global/u1/g/gelijerg/Projects/pyinsulate


In [2]:
from datetime import datetime
import torch

from experiments.A_proof_of_constraint.experiment_definition import dictionary_product
from experiments.A_proof_of_constraint.main import run_experiment

In [3]:
# Saving utilities
def get_savefile():
    base_name = "proof-of-constraint"
    time_string = datetime.now().strftime('%Y-%m-%d-%H-%M-%S')
    savefile = f"{base_name}_{time_string}.pth"
    return savefile


def save_out(summary, savefile, directory="/global/u1/g/gelijerg/Projects/pyinsulate/results"):
    full_file = f"{directory}/{savefile}"
    print(f"Saving to file {full_file}")
    torch.save(summary, full_file)

In [4]:
base_configuration = {
    'training_sampling': "uniform",
    'num_points': 1000,
    'num_training': 500,
    'batch_size': 100,
    'model_size': [20, 20, 20],
    'method': "unconstrained",
    'learning_rate': 1e-3, # 1e-2 works for unconstrained well
}

configuration_revisions = list(dictionary_product(**{
#     'method': ["unconstrained", "average", "batchwise", "no-loss"],
    'method': ["batchwise"],
#     'learning_rate': [1e-2, 1e-3, 1e-4, 1e-5],
}))

configurations = list()
for revision in configuration_revisions:
    configurations.append(base_configuration.copy())
    configurations[-1].update(revision)
num_epochs = 20

In [5]:
# Run experiment
all_savefiles = list()
for configuration in configurations:
    savefile = get_savefile()
    all_savefiles.append(savefile)
    print(f"Running proof of constraint with savefile {savefile}")
    final_result = run_experiment(num_epochs, log=print, **configuration)
    print(f"Completed run with savefile {savefile}")
    # Save out
    configuration, (trainer, train_evaluator, test_evaluator), \
    (training_monitor, evaluation_train_monitor, evaluation_test_monitor) = final_result

    save_out({
        'configuration': configuration,
        'training_monitor': training_monitor,
        'evaluation_train_monitor': evaluation_train_monitor,
        'evaluation_test_monitor': evaluation_test_monitor,
    }, savefile=savefile)
print(f"\nFiles were saved to {all_savefiles}")

Running proof of constraint with savefile proof-of-constraint_2019-07-25-09-40-10.pth
{'frequency': 1.0, 'phase': None, 'amplitude': 1.0, 'num_points': 1000, 'num_training': 500, 'training_sampling': 'uniform', 'batch_size': 100, 'model_size': [20, 20, 20], 'model_act': ReLU(), 'model_final_act': None, 'learning_rate': 0.001, 'method': 'batchwise'}
Error occurred while computing constrained loss:
cholesky_cpu: U(12,12) is zero, singular U.
Constraints are likely ill-conditioned (i.e. jacobian is not full rank at this point). Falling back to computing pseudoinverse
Epoch[1] - Constrained loss: 0.53982, Loss: 0.46769
Error occurred while computing constrained loss:
cholesky_cpu: U(17,17) is zero, singular U.
Constraints are likely ill-conditioned (i.e. jacobian is not full rank at this point). Falling back to computing pseudoinverse
Epoch[1] - Constrained loss: 0.51794, Loss: 0.46792
Error occurred while computing constrained loss:
cholesky_cpu: U(27,27) is zero, singular U.
Constraints 

In [6]:
print('done!')

done!
